# Lab 4 - OpenAI Agents SDK + TOOLS!!

In [31]:
from dotenv import load_dotenv
from agents import Agent, Runner, function_tool
from openai.types.responses import ResponseTextDeltaEvent
import gradio as gr
import os
import requests

load_dotenv(override=True)

True

In [32]:
pushover_user = os.getenv("PUSHOVER_USER")
pushover_token = os.getenv("PUSHOVER_TOKEN")
pushover_url = "https://api.pushover.net/1/messages.json"

In [33]:
def send_push_notification(message: str):
    payload = {"user": pushover_user, "token": pushover_token, "message": message}
    requests.post(pushover_url, data=payload)

In [34]:
send_push_notification("Hello, me!")

In [35]:
@function_tool
def push(message: str) -> str:
    """Send a text message as a push notification to Oscar with this brief message

    Args:
        message: The short text message to push to Oscar
    """

    send_push_notification(message)
    return "Push notification sent"

In [36]:
push.params_json_schema

{'properties': {'message': {'description': 'The short text message to push to Oscar',
   'title': 'Message',
   'type': 'string'}},
 'required': ['message'],
 'title': 'push_args',
 'type': 'object',
 'additionalProperties': False}

In [37]:
instructions = """
You represent the AI Digital Twin of a human called Oscar.
You are friendly and amiable, and you introduce yourself as Oscars's Digital Twin.
Oscar is the co-founder and CTO of AI startup Nebula.io.
He loves coding and experimenting with LLMs, and he speaks at conferences and lectures about LLMs.
You chat with visitors on Oscar's personal website. You answer questions about Oscar's work.
If you don't know the answer, send a push notification to Oscar to tell him the question you couldn't answer, so that he adds it to the knowledge base.
"""

In [38]:
agent = Agent(name="Twin", instructions=instructions, model="gpt-4.1-mini", tools=[push])

In [39]:
async def chat(message, history):
    messages = [{"role": prior["role"], "content": prior["content"]} for prior in history]
    messages += [{"role": "user", "content": message}]
    response = Runner.run_streamed(agent, messages)
    reply = ""
    async for event in response.stream_events():
        if event.type == "raw_response_event" and isinstance(event.data, ResponseTextDeltaEvent):
            reply += event.data.delta
            yield reply

In [40]:
gr.ChatInterface(chat, type="messages").launch()

* Running on local URL:  http://127.0.0.1:7860
* To create a public link, set `share=True` in `launch()`.
